In [1]:


from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image


In [2]:

model = AutoModelForCausalLM.from_pretrained(
    "vikhyatk/moondream2",
    revision="2025-04-14",
    trust_remote_code=True,
    # Uncomment to run on GPU.
    device_map={"": "cuda"}
)

moondream.py:   0%|          | 0.00/26.4k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [18]:
model.model.region

ModuleDict(
  (coord_encoder): Linear(in_features=256, out_features=2048, bias=True)
  (coord_decoder): ModuleDict(
    (fc1): Linear(in_features=2048, out_features=8192, bias=True)
    (fc2): Linear(in_features=8192, out_features=1024, bias=True)
  )
  (size_encoder): Linear(in_features=512, out_features=2048, bias=True)
  (size_decoder): ModuleDict(
    (fc1): Linear(in_features=2048, out_features=8192, bias=True)
    (fc2): Linear(in_features=8192, out_features=2048, bias=True)
  )
)

In [14]:
model.model.config.tokenizer.templates['detect']

{'prefix': [198, 198, 47504, 25], 'suffix': [628]}

In [19]:
model.model.tokenizer

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[{"id":50256, "content":"<|endoftext|>", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":50257, "content":"                               ", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":50258, "content":"                              ", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":50259, "content":"                             ", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":50260, "content":"                            ", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":50261, "content":"                           ", "single_word":False, "lstrip":False, "rstrip":False, "normalized":True, "special":False}, {"id":50262, "content":"                          ", "single_word":Fal

In [16]:
model.model.tokenizer.decode([198, 198, 47504, 25])

'\n\nDetect:'

In [17]:
model.model.tokenizer.decode([628])

'\n\n'

In [5]:
model.model.detect??

Signature:
model.model.detect(
    image: Union[PIL.Image.Image, transformers_modules.vikhyatk.moondream2.797e1e4728e49ce676920e72e42cd0cb3948f504.moondream.EncodedImage],
    object: str,
    settings: Optional[transformers_modules.vikhyatk.moondream2.797e1e4728e49ce676920e72e42cd0cb3948f504.moondream.ObjectSamplingSettings] = None,
)
Docstring: <no docstring>
Source:   
    def detect(
        self,
        image: Union[Image.Image, EncodedImage],
        object: str,
        settings: Optional[ObjectSamplingSettings] = None,
    ):
        if self.config.tokenizer.templates["detect"] is None:
            raise NotImplementedError("Model does not support object detection.")

        image = self.encode_image(image)
        self.load_encoded_image(image)

        prompt_tokens = torch.tensor(
            [
                self.config.tokenizer.templates["detect"]["prefix"]
                + self.tokenizer.encode(" " + object).ids
                + self.config.tokenizer.templates["dete

In [8]:
model.model._generate_points??

Signature:
model.model._generate_points(
    hidden: torch.Tensor,
    next_token: torch.Tensor,
    pos: int,
    include_size: bool = True,
    max_objects: int = 50,
)
Docstring: <no docstring>
Source:   
    def _generate_points(
        self,
        hidden: torch.Tensor,
        next_token: torch.Tensor,
        pos: int,
        include_size: bool = True,
        max_objects: int = DEFAULT_MAX_OBJECTS,
    ):
        out = []
        mask = torch.zeros(1, 1, 2048, device=self.device, dtype=torch.bool)
        mask[:, :, :pos] = 1
        pos_ids = torch.tensor([pos], device=self.device, dtype=torch.long)

        with torch.inference_mode():
            while (
                next_token.item() != self.config.tokenizer.eos_id
                and len(out) < max_objects
            ):
                x_logits = decode_coordinate(hidden, self.region)
                x_center = torch.argmax(x_logits, dim=-1) / x_logits.size(-1)
                next_emb = encode_coordinate(
         